<a href="https://colab.research.google.com/github/aliaghababaee/SLO_Asieh/blob/main/Copy_of_Optuna_merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive'

/content/drive/MyDrive


In [ ]:
!pip install optuna
!pip install optuna-integration
import optuna
from optuna.trial import TrialState
from optuna.integration import TFKerasPruningCallback

In [ ]:
####################### state of the arts ##############################

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout
import tensorflow as tf
from keras.layers import Input

def classifier (trial):

    ### SLO model
    slo_model =  tf.keras.applications.VGG19(
        weights='imagenet',
        include_top=False,
        input_shape=(128,128,3),
        )

    slo_model.get_layer(index = 0)._name = 'SLO'

    for layer in slo_model.layers:
        layer.trainable = False

    ### OCT model

    oct_model =  tf.keras.applications.VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(60,256,3),
        )

    oct_model.get_layer(index = 0)._name = 'OCT'

    for layer in oct_model.layers:
        layer.trainable = False


    inputs_slo = tf.keras.layers.Input((128,128,3))
    slo_output = tf.keras.layers.Flatten()(slo_model(inputs_slo))


    inputs_oct = tf.keras.layers.Input((60,256,3))
    oct_output = tf.keras.layers.Flatten()(oct_model(inputs_oct))


    model = tf.keras.layers.Concatenate(axis=-1)([slo_output, oct_output])


    dropout_l0 = trial.suggest_float("dropout_l0", 0, 0.7,step=0.1)

    model = tf.keras.layers.Dropout(dropout_l0)(model)


    n_layers = trial.suggest_int("n_layers", 1, 5)


    for i in range(n_layers):

        n_units = trial.suggest_int("n_units_l{}".format(i), 8, 4096, log = True)

        model = tf.keras.layers.Dense(n_units, activation = 'relu')(model)

        dropout = trial.suggest_float("dropout_l{}".format (i), 0, 0.7,step=0.1)

        model = tf.keras.layers.Dropout(rate = dropout)(model)


    outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(model)

    lr = trial.suggest_float ('lr', 1e-5, 1e-3, log = True)

    my_optimizer = tf.keras.optimizers.Adam (learning_rate= lr)

    model_merged = tf.keras.Model([inputs_slo, inputs_oct] , outputs)


    model_merged.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics='accuracy')

    return model_merged

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold (n_splits = 5, shuffle = True, random_state = 42)
import keras
import pickle
from keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


def preparing(x, y):

    data_slo  = []
    data_oct  = []
    label     = []
    for i in x:
      for j in range(len(x[i])):
          data_slo.append(np.array(x[i][j][0])/255)
          data_oct.append(np.array(x[i][j][1])*255)
          label.append(y[i])
    data_slo = np.reshape(data_slo, np.shape(data_slo))
    data_oct = np.reshape(data_oct, np.shape(data_oct))
    return data_slo, data_oct, np.array(label)



def objective (trial):

  keras.backend.clear_session()

##################??????????????????????????????????????????????

  datagen_oct = ImageDataGenerator(
  rotation_range= 5, # rotation
  zoom_range= 0.2,
  vertical_flip= True , # vertical fli
  fill_mode='constant',
  data_format='channels_last',
  cval=0,
    )

  datagen_slo = ImageDataGenerator(
  rotation_range= 5, # rotation
  width_shift_range= [-30, 30], # horizontal shift
  height_shift_range= [-5, 5] , # vertical shift
  zoom_range= 0.2,
  vertical_flip= True , # vertical flip
  brightness_range= [0.2, 1.5]
    )


  images_train = pickle.load(open("train_merged.pkl", 'rb'))
  labels_train = pickle.load(open("train_merged_label.pkl", 'rb'))

  train_index, val_index = next (skf.split (images_train, list(labels_train.values())))

  x_train = {i: images_train[list(images_train.keys())[i]] for i in train_index}
  x_valid = {i: images_train[list(images_train.keys())[i]] for i in val_index}

  y_trainn = {i: labels_train[list(labels_train.keys())[i]] for i in train_index}
  y_validd = {i: labels_train[list(labels_train.keys())[i]] for i in val_index}


  ################## preparing

  x_train_slo, x_train_oct, y_train = preparing(x_train,y_trainn)
  x_valid_slo, x_valid_oct, y_valid = preparing(x_valid,y_validd)

  ################# Augmentation

  ############ slo
  x_train_aug =np.zeros_like(x_train_slo, dtype=np.float32)

  y_train_aug =np.zeros_like(y_train, dtype=np.float32)


  for i in range(len(x_train_slo)):

    x1= x_train_slo[i,:,:,:].copy()

    x1=x1.reshape((1, ) + x1.shape)

    x = datagen_slo.flow(x1, batch_size=1, seed=42) # to make the result reproducible

    x_train_aug [i,:,:,:] = x.next()

    y_train_aug [i] = y_train[i]

  x_train_slo = np.concatenate ((x_train_slo, x_train_aug), axis=0)

  y_train_slo = np.concatenate ((y_train, y_train_aug), axis = 0)

  ############ OCT
  x_train_aug =np.zeros_like(x_train_oct, dtype=np.float32)

  y_train_aug =np.zeros_like(y_train, dtype=np.float32)

  for i in range(len(x_train_oct)):

    x1= x_train_oct[i,:,:,:].copy()

    x1=x1.reshape((1, ) + x1.shape)

    x = datagen_oct.flow(x1, batch_size=1, seed=42) # to make the result reproducible

    x_train_aug [i,:,:,:] = x.next()

    y_train_aug [i] = y_train[i]

  x_train_oct = np.concatenate ((x_train_oct, x_train_aug), axis=0)

  y_train_oct = np.concatenate ((y_train, y_train_aug), axis = 0)


  indices = np.random.permutation (len (x_train_slo))
  x_train_slo = x_train_slo [indices]
  y_train_slo = y_train_slo [indices]

  x_train_oct = x_train_oct [indices]
  y_train_oct = y_train_oct [indices]


  x_train_slo = np.repeat (x_train_slo, repeats = 3, axis = 3)

  x_train_oct = np.repeat (x_train_oct, repeats = 3, axis = 3)

  x_valid_slo = np.repeat (x_valid_slo, repeats = 3, axis = 3)

  x_valid_oct = np.repeat (x_valid_oct, repeats = 3, axis = 3)


  ####################################################################
  # classification via my model
  ####################################################################
  batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64, 128])

  model = classifier (trial)


  # Generate our trial model.


  model.fit([x_train_slo, x_train_oct],
            np.asarray(y_train_oct, dtype=np.float64),
            batch_size= batch_size,
            epochs=50,
            callbacks=[TFKerasPruningCallback(trial, "val_accuracy"), #EarlyStopping(patience=10, verbose=1),
            ReduceLROnPlateau(factor=0.1, patience=10, min_lr=1e-6),
            ModelCheckpoint(f'slo_oct.h5', verbose=1, save_best_only=True, save_weights_only=True)],
            validation_data=([x_valid_slo, x_valid_oct] , np.asarray(y_valid, dtype=np.float64)),
            )
  model.load_weights(f'slo_oct.h5')
  score = model.evaluate ([x_valid_slo, x_valid_oct], np.asarray(y_valid, dtype=np.float64), verbose = 1)

  return score [1]


In [ ]:
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.SuccessiveHalvingPruner())

study.optimize(objective, n_trials=100)

[I 2024-02-28 08:38:32,458] A new study created in memory with name: no-name-323bdc79-3b6d-4885-b225-be0c50142965


Epoch 1/50
11/11 [==============================] - ETA: 0s - loss: 4.7669 - accuracy: 0.5296
Epoch 1: val_loss improved from inf to 0.58922, saving model to slo_oct.h5
11/11 [==============================] - 19s 1s/step - loss: 4.7669 - accuracy: 0.5296 - val_loss: 0.5892 - val_accuracy: 0.6047 - lr: 4.2395e-04
Epoch 2/50
11/11 [==============================] - ETA: 0s - loss: 4.3263 - accuracy: 0.4970
Epoch 2: val_loss improved from 0.58922 to 0.35785, saving model to slo_oct.h5
11/11 [==============================] - 4s 401ms/step - loss: 4.3263 - accuracy: 0.4970 - val_loss: 0.3578 - val_accuracy: 0.9535 - lr: 4.2395e-04
Epoch 3/50
11/11 [==============================] - ETA: 0s - loss: 2.6656 - accuracy: 0.5858
Epoch 3: val_loss did not improve from 0.35785
11/11 [==============================] - 1s 112ms/step - loss: 2.6656 - accuracy: 0.5858 - val_loss: 0.4051 - val_accuracy: 0.9070 - lr: 4.2395e-04
Epoch 4/50
11/11 [==============================] - ETA: 0s - loss: 2.4458 

[I 2024-02-28 08:41:03,247] Trial 0 finished with value: 0.9069767594337463 and parameters: {'batch_size': 32, 'dropout_l0': 0.7, 'n_layers': 3, 'n_units_l0': 417, 'n_units_l1': 87, 'dropout_l1': 0.30000000000000004, 'n_units_l2': 48, 'dropout_l2': 0.4, 'lr': 0.0004239450247224403}. Best is trial 0 with value: 0.9069767594337463.


Epoch 1/50
6/6 [==============================] - ETA: 0s - loss: 0.7085 - accuracy: 0.4941
Epoch 1: val_loss improved from inf to 0.68395, saving model to slo_oct.h5
6/6 [==============================] - 26s 2s/step - loss: 0.7085 - accuracy: 0.4941 - val_loss: 0.6840 - val_accuracy: 0.4651 - lr: 1.8553e-05
Epoch 2/50
6/6 [==============================] - ETA: 0s - loss: 0.6965 - accuracy: 0.5118
Epoch 2: val_loss improved from 0.68395 to 0.67442, saving model to slo_oct.h5
6/6 [==============================] - 2s 377ms/step - loss: 0.6965 - accuracy: 0.5118 - val_loss: 0.6744 - val_accuracy: 0.5349 - lr: 1.8553e-05
Epoch 3/50
6/6 [==============================] - ETA: 0s - loss: 0.6692 - accuracy: 0.5799
Epoch 3: val_loss improved from 0.67442 to 0.64749, saving model to slo_oct.h5
6/6 [==============================] - 2s 373ms/step - loss: 0.6692 - accuracy: 0.5799 - val_loss: 0.6475 - val_accuracy: 0.7442 - lr: 1.8553e-05
Epoch 4/50
6/6 [==============================] - ETA: 

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  73
  Number of pruned trials:  21
  Number of complete trials:  51
Best trial:
  Value:  0.7096773982048035
  Params: 
    batch_size: 64
    n_layers: 3
    n_units_l0: 591
    dropout_l0: 0.30000000000000004
    n_units_l1: 19
    dropout_l1: 0.0
    n_units_l2: 182
    dropout_l2: 0.0
    lr: 0.00011541182649610528
